In [1]:
from py2neo import Graph
from bs4 import BeautifulSoup
import pywikibot
import pandas as pd
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
import wikipedia
import spacy
nlp = spacy.load("en_core_web_sm")
import gensim
from collections import defaultdict
from gensim.parsing.preprocessing import strip_multiple_whitespaces, preprocess_string, remove_stopwords, strip_tags, strip_punctuation 

In [2]:
graph = Graph(password="kshen3778")

In [3]:
graph.run("MATCH (n:Category) RETURN n LIMIT 25").data()

[{'n': (_5214855:Category:Page {id: 690070, isNew: false, isRedirect: false, title: 'Futurama'})},
 {'n': (_5214856:Category:Page {id: 690451, isNew: false, isRedirect: false, title: 'World_War_II'})},
 {'n': (_5214857:Category:Page {id: 690571, isNew: false, isRedirect: false, title: 'Programming_languages'})},
 {'n': (_5214858:Category:Page {id: 690578, isNew: false, isRedirect: false, title: 'Professional_wrestling'})},
 {'n': (_5214859:Category:Page {id: 690637, isNew: false, isRedirect: false, title: 'Algebra'})},
 {'n': (_5214860:Category:Page {id: 690649, isNew: false, isRedirect: false, title: 'Anime'})},
 {'n': (_5214861:Category:Page {id: 690672, isNew: false, isRedirect: false, title: 'Abstract_algebra'})},
 {'n': (_5214862:Category:Page {id: 690747, isNew: false, isRedirect: false, title: 'Mathematics'})},
 {'n': (_5214863:Category:Page {id: 690777, isNew: false, isRedirect: false, title: 'Linear_algebra'})},
 {'n': (_5214864:Category:Page {id: 690803, isNew: false, isRedir

In [20]:
graph.run("MATCH (n:Category { title: 'Cold_War' }) RETURN n").data()

[{'n': (_5214918:Category:Page {id: 691614, isNew: false, isRedirect: false, title: 'Cold_War'})}]

In [55]:
#get all sub cat
cats = graph.run("MATCH (n:Category{title:'Constraint_logic_programming'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()

In [56]:
len(cats)

2

In [57]:
cats

[{'p': (_6124108:Category:Page {id: 22968506, isNew: false, isRedirect: false, title: 'Constraint_programming'})},
 {'p': (_5594005:Category:Page {id: 1188828, isNew: false, isRedirect: false, title: 'Logic_programming'})}]

In [27]:
getBadCategories("Artificial_intelligence")

['Areas_of_computer_science',
 'Cognitive_science',
 'Computational_neuroscience',
 'Cybernetics',
 'Emerging_technologies',
 'Formal_sciences',
 'Futurology',
 'Intelligence_by_type',
 'Personhood',
 'Technology_in_society',
 'Unsolved_problems_in_computer_science']

In [63]:
#check if a specific sub category references the select category an two way relationship (which we don't want)
subs = graph.run("MATCH (n:Category{title:'Emerging_technologies'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()

In [64]:
subs

[]

In [2]:
#verify if category actually exists on online Wikipedia
def categoryExist(name):
    res = requests.get("https://en.wikipedia.org/wiki/Category:" + name)
    soup = BeautifulSoup(res.content, "html.parser")
    div = soup.find("div", {"id": "mw-normal-catlinks"})
    if(div):
        return True
    else:
        return False

#Use beautifulsoup to scrape the bottom categories of a parent category (these we don't want)
def getBadCategories(parent):
    res = requests.get("https://en.wikipedia.org/wiki/Category:" + parent)
    soup = BeautifulSoup(res.content, "html.parser")
    div = soup.find("div", {"id": "mw-normal-catlinks"})
    if(div):
        to_return = []
        for item in div.findAll("li"):
            to_return.append("_".join(item.get_text().split(" ")))
        return to_return
    else:
        #parent does not exist
        return []

#get all immediate Wikipedia subcategories of main minus two way connections

# def getAllSubCat(parent):
#     all_cats = graph.run("MATCH (n:Category{title:'" + parent + "'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()
#     bad_cats = getBadCategories(parent)
#     filtered = []
#     for category in all_cats:
#         cat_title = category['p']["title"]
#         if (not (cat_title in bad_cats)):
#             filtered.append(cat_title)
#     #search all these categories to see if any of them match the other way (if so, we remove)
# #     filtered = []
# #     for category in all_cats:
# #         cat_title = category['p']["title"]
# #         subs = graph.run("MATCH (n:Category{title:'" + cat_title + "'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()
# #         found = False
# #         for category_2 in subs:
# #             sub_title = category_2['p']["title"]
# #             if(sub_title == parent):
# #                 found = True
# #         if (not found):
# #             filtered.append(cat_title)
    
#     return filtered

def getSubCategories(name):
    site = pywikibot.Site()
    gen = pywikibot.Category(site,'Category:' + name).subcategories(recurse=False)
    subcats = []
    for item in gen:
        subcats.append("_".join(item.aslink().split(":")[1][:-2].split(" ")))
    return subcats  

In [11]:
getAllSubCat("Works_about_personality")
#getBadCategories("Constraint_logic_programming")

['Borderline_personality_disorder_in_fiction',
 'Histrionic_personality_disorder_in_fiction']

In [3]:
blacklist = ["July_events"] #nodes that should not be visited
all_nodes = []
all_relations = [] #child parent
visited = []
def buildGraph(root, path):
    path = path + "/" + root
    print("Current Path: " + path)
    print()
    if((not categoryExist(root)) or (root in blacklist)):
        print("Category DNE or is BLACKLISTED")
        print()
        return
    if(root in visited):
        print("VISITED ALREADY")
        print()
        return
    
    visited.append(root)
    
    #add the node
    all_nodes.append(root)
    
    #get all subcategories
    sub_cat = getSubCategories(root)
    if(len(sub_cat) == 0): #base case
        return

    
    #add all the relationships
    for item in sub_cat:
        #check if this relationship already exists (cycle)
        relation = item + " " + root
        if(relation in all_relations):
            print("Relation already exists: ", relation)
            print()
            return
        all_relations.append(item + " " + root)
   
    #recurse on all sub nodes
    for item in sub_cat:
        buildGraph(item, path)


In [4]:
buildGraph("Machine_learning", "")

Current Path: /Machine_learning

Current Path: /Machine_learning/Applied_machine_learning

Current Path: /Machine_learning/Applied_machine_learning/AlphaGo

Current Path: /Machine_learning/Artificial_neural_networks

Current Path: /Machine_learning/Artificial_neural_networks/Deep_learning

Current Path: /Machine_learning/Artificial_neural_networks/Neural_network_software

Current Path: /Machine_learning/Bayesian_networks

Current Path: /Machine_learning/Classification_algorithms

Current Path: /Machine_learning/Classification_algorithms/Artificial_neural_networks

VISITED ALREADY

Current Path: /Machine_learning/Classification_algorithms/Decision_trees

Current Path: /Machine_learning/Classification_algorithms/Ensemble_learning

Current Path: /Machine_learning/Cluster_analysis

Current Path: /Machine_learning/Cluster_analysis/Cluster_analysis_algorithms

Current Path: /Machine_learning/Cluster_analysis/Clustering_criteria

Current Path: /Machine_learning/Computational_learning_theory



In [12]:
all_nodes

['Machine_learning',
 'Applied_machine_learning',
 'AlphaGo',
 'Artificial_neural_networks',
 'Deep_learning',
 'Neural_network_software',
 'Bayesian_networks',
 'Classification_algorithms',
 'Decision_trees',
 'Ensemble_learning',
 'Cluster_analysis',
 'Cluster_analysis_algorithms',
 'Clustering_criteria',
 'Computational_learning_theory',
 'Artificial_intelligence_conferences',
 'Signal_processing_conferences',
 'Data_mining_and_machine_learning_software',
 'Social_network_analysis_software',
 'Datasets_in_machine_learning',
 'Datasets_in_computer_vision',
 'Dimension_reduction',
 'Factor_analysis',
 'Evolutionary_algorithms',
 'Gene_expression_programming',
 'Genetic_algorithms',
 'Artificial_immune_systems',
 'Genetic_programming',
 'Nature-inspired_metaheuristics',
 'Inductive_logic_programming',
 'Kernel_methods_for_machine_learning',
 'Support_vector_machines',
 'Latent_variable_models',
 'Structural_equation_models',
 'Learning_in_computer_vision',
 'Log-linear_models',
 'Loss_

In [6]:
len(all_relations)

61

In [13]:
#wikipedia.search("Herbertien_dynasty")

ConnectionError: HTTPConnectionPool(host='en.wikipedia.org', port=80): Max retries exceeded with url: /w/api.php?prop=info%7Cpageprops&inprop=url&ppprop=disambiguation&redirects=&titles=Elon+Musk&format=json&action=query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fef00090320>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [19]:
try:
    p = wikipedia.page('Pepin, Count of Vermandois')
    print(p.content)
except wikipedia.DisambiguationError as e:
    print(e.options[0])
except Exception:
    print("Caught")

Caught


In [15]:
#save graph to files
fileName = "data/ML"
with open(fileName + '_nodes.txt', 'w') as f:
    for item in all_nodes:
        f.write("%s\n" % item)

with open(fileName + '_relations.txt', 'w') as f:
    for item in all_relations:
        f.write("%s\n" % item)        

In [16]:
def preprocess(x):
    x = gensim.utils.simple_preprocess(x)
    x = " ".join(x)
    
    CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, remove_stopwords]
    x = preprocess_string(x, CUSTOM_FILTERS)
    return x

In [18]:
#Extract wikipedia articles (to convert into feature vectors)
file = open(fileName + '_nodes.txt', "r")
articles_cleaned = []
freq_threshold = 3
for line in file:
    print("==" + line + "==")
    search = wikipedia.search(line.rstrip("\n"))[0]
    try:
        page = wikipedia.page(search)
    except wikipedia.DisambiguationError as e:
        page = wikipedia.page(e.options[0])
    except Exception:
        articles_cleaned.append("")
        continue;
    #page = wikipedia.page(wikipedia.search(line.rstrip("\n"))[0])
    content = page.content
    cleaned = preprocess(content) #strip punctuations and stopwords and weird characters
    cleaned = " ".join(cleaned)
    doc = nlp(cleaned)
    lemm = [token.lemma_ for token in doc]
    d = defaultdict(int)
    for item in lemm:
        d[item] += 1
    tokens=[key for key,value in d.items() if value>freq_threshold] #all words with frequency of more than some number
    texts = [word for word in lemm if word in tokens]
    texts = " ".join(texts)
    articles_cleaned.append(texts)
    #print(texts)
file.close()

==Machine_learning
==
==Applied_machine_learning
==
==AlphaGo
==
==Artificial_neural_networks
==
==Deep_learning
==
==Neural_network_software
==
==Bayesian_networks
==
==Classification_algorithms
==
==Decision_trees
==
==Ensemble_learning
==
==Cluster_analysis
==
==Cluster_analysis_algorithms
==
==Clustering_criteria
==
==Computational_learning_theory
==
==Artificial_intelligence_conferences
==
==Signal_processing_conferences
==
==Data_mining_and_machine_learning_software
==


WikipediaException: An unknown error occured: "Search is currently too busy. Please try again later.". Please report it on GitHub!

In [21]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(articles_cleaned)
print(vectorizer.get_feature_names())
print(vectors.shape)

['aachen', 'ab', 'abandon', 'abbandonata', 'abbasid', 'abbey', 'abbie', 'abbot', 'abdel', 'abdicate', 'abduct', 'ability', 'ablative', 'able', 'aboard', 'abolish', 'abolition', 'aboriginal', 'aborigine', 'abraham', 'abroad', 'absalom', 'absent', 'absolute', 'abuse', 'abydos', 'academic', 'academica', 'academy', 'acarnanian', 'accept', 'access', 'accession', 'accommodate', 'accommodation', 'accompagnato', 'accompany', 'accord', 'accordingly', 'account', 'accretion', 'accusation', 'accusative', 'accuse', 'acerba', 'achaea', 'achaean', 'achaemenid', 'achieve', 'achievement', 'achilla', 'achille', 'acidalia', 'acidinus', 'acknowledge', 'aconia', 'acquaintance', 'acquire', 'acquisition', 'acquit', 'acra', 'acre', 'acrocorinth', 'acropoli', 'acropolis', 'act', 'action', 'actium', 'active', 'activity', 'actor', 'actress', 'actually', 'ad', 'adaloald', 'adam', 'adapt', 'adaptation', 'add', 'addison', 'addition', 'additional', 'additionally', 'address', 'adelchi', 'adjacent', 'administration', 

In [3]:
article_names = []
file = open(fileName + '_nodes.txt', "r")
for line in file:
    article_names.append(line.rstrip("\n"))
file.close()

In [4]:
article_names

['Roman_Republic',
 'Ancient_Roman_Republican_art',
 'Government_of_the_Roman_Republic',
 'Ancient_Roman_governors',
 'Ancient_Roman_governors_by_province',
 'Lists_of_Roman_governors',
 'Roman_governors_of_Achaea',
 'Roman_governors_of_Africa',
 'Roman_governors_of_Arabia_Petraea',
 'Roman_governors_of_Asia',
 'Roman_governors_of_Bithynia_and_Pontus',
 'Roman_governors_of_Britain',
 'Roman_governors_of_Campania',
 'Roman_governors_of_Cappadocia',
 'Roman_governors_of_Cilicia',
 'Roman_governors_of_Crete_and_Cyrenaica',
 'Roman_governors_of_Cyprus',
 'Roman_governors_of_Dacia',
 'Roman_governors_of_Dalmatia',
 'Roman_governors_of_Egypt',
 'Roman_governors_of_Galatia',
 'Roman_governors_of_Gallia_Aquitania',
 'Roman_governors_of_Gallia_Belgica',
 'Roman_governors_of_Gallia_Lugdunensis',
 'Roman_governors_of_Gallia_Narbonensis',
 'Roman_governors_of_Gaul',
 'Praetorian_prefects_of_Gaul',
 'Roman_governors_of_Germania_Inferior',
 'Roman_governors_of_Germania_Superior',
 'Roman_governors_o

In [24]:
vectors

<1121x7034 sparse matrix of type '<class 'numpy.float64'>'
	with 86605 stored elements in Compressed Sparse Row format>

In [25]:
df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())

In [26]:
df["article_name"] = article_names

In [27]:
#save to a file
df.to_hdf(fileName + "_embeddings.h5", key='df')

In [6]:
#read from file
data_table = pd.read_hdf(fileName + "_embeddings.h5", 'df')

In [12]:
data_table["article_name"]
data_table.index[data_table['article_name'] == "Julius_Caesar"]

Int64Index([316, 842, 887], dtype='int64')

In [2]:
data_table = pd.read_hdf(fileName + "_embeddings.h5", 'df')

In [4]:
data_table.loc[1, "article_name"]

'Artificial_neural_networks'

In [5]:
data_table["article_name"]

0                              Machine_learning
1                    Artificial_neural_networks
2                       Neural_network_software
3                                 Deep_learning
4                      Applied_machine_learning
5                                       AlphaGo
6                         Unsupervised_learning
7                         Structured_prediction
8                              Graphical_models
9                             Bayesian_networks
10                             Causal_inference
11                              Markov_networks
12                               Loss_functions
13                      Support_vector_machines
14                       Latent_variable_models
15                   Structural_equation_models
16                              Factor_analysis
17                             Cluster_analysis
18                          Clustering_criteria
19                  Cluster_analysis_algorithms
20                Signal_processing_conf

In [6]:
file = open(fileName + "_relations.txt", "r")
for line in file:
    print(line.rstrip("\n"))
file.close()

Artificial_neural_networks Machine_learning
Applied_machine_learning Machine_learning
Unsupervised_learning Machine_learning
Structured_prediction Machine_learning
Loss_functions Machine_learning
Support_vector_machines Machine_learning
Latent_variable_models Machine_learning
Cluster_analysis Machine_learning
Signal_processing_conferences Machine_learning
Artificial_intelligence_conferences Machine_learning
Data_mining_and_machine_learning_software Machine_learning
Machine_learning_algorithms Machine_learning
Bayesian_networks Machine_learning
Machine_learning_portal Machine_learning
Statistical_natural_language_processing Machine_learning
Markov_models Machine_learning
Evolutionary_algorithms Machine_learning
Genetic_programming Machine_learning
Datasets_in_machine_learning Machine_learning
Dimension_reduction Machine_learning
Machine_learning_researchers Machine_learning
Supervised_learning Machine_learning
Semisupervised_learning Machine_learning
Computational_learning_theory Machin